<a href="https://colab.research.google.com/github/alinemmc/UFU_PPGEQ_AI_AlineCardoso/blob/main/ATIVIDADE_2_IDENTIFICA%C3%87%C3%83O_DE_PLACAS_ALINE_MARIA_DE_JESUS_MACHADO_CARDOSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [359]:
!pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [360]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [361]:
import cv2
from google.colab.patches import cv2_imshow
from skimage import feature
import numpy as np
import pytesseract
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
from google.colab import files
!pip install xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 3.8 MB/s eta 0:00:00


In [362]:
!gdown 17DeutH_Rzmw7J6Qdr9XdMFTRSH8v7VvY

Downloading...
From: https://drive.google.com/uc?id=17DeutH_Rzmw7J6Qdr9XdMFTRSH8v7VvY
To: /content/licence_plate.zip
100% 2.96M/2.96M [00:00<00:00, 164MB/s]


In [363]:
PLACASDECARROS ='licence_plate.zip'
archive = zipfile.ZipFile(PLACASDECARROS, 'r')
archive.extractall()

In [364]:
img_name=[]
with zipfile.ZipFile(PLACASDECARROS, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [365]:
carro = []
placa =[]

In [366]:
for word in img_name:
    img = cv2.imread(word)
    
    #cv2_imshow(img)
    cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    test, bin = cv2.threshold(cinza, 100, 255, cv2.THRESH_BINARY)
    desfoque = cv2.GaussianBlur(bin, (5, 5), 0)
    contornos, hierarquia = cv2.findContours(desfoque, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(img, contornos, -1, (90, 255, 0), 1)
    for c in contornos:
        perimetro = cv2.arcLength(c, True)
        if perimetro > 120:
            aprox = cv2.approxPolyDP(c, 0.03 * perimetro, True)
            if len(aprox) == 4:
                (x, y, alt, lar) = cv2.boundingRect(c)
                cv2.rectangle(img, (x, y), (x + alt, y + lar), (0, 255, 0), 2)
                roi = img[y:y + lar, x:x + alt]
    resize_img_roi = cv2.resize(roi, None, fx=4, fy=4, interpolation=cv2.INTER_CUBIC)
    img_cinza = cv2.cvtColor(resize_img_roi, cv2.COLOR_BGR2GRAY)
    _, img_binary = cv2.threshold(img_cinza, 70, 255, cv2.THRESH_BINARY)
    img_desfoque = cv2.GaussianBlur(img_binary, (5, 5), 0)
    img_ocr = img_desfoque
    #cv2_imshow(img)
    config = r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 --psm 6'
    
    texto = pytesseract.image_to_string(img_ocr, lang='eng', config=config)
    placa.append(texto)
    carro.append(word)


In [367]:
df = pd.DataFrame({'Imagem':carro,'Placa':placa})
df.to_excel('Placas.xlsx')
files.download('Placas.xlsx')
df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Imagem,Placa
0,placa_carro3.jpg,EE\n
1,placa_carro2.jpg,
2,00011.jpg,
3,00024.jpg,ON\nL\nP\n
4,00029.jpg,4\nP\n
5,car_0001.jpg,SS\n
6,car_0002.jpg,
7,car_0004.jpg,17\n1\n
8,car_0005.jpg,C\n
9,car_0006.jpg,C\n
